In [50]:
import cv2
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
from ultralytics import YOLO, solutions

model = YOLO("yolov8l.pt")
cap = cv2.VideoCapture("video.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define line points
line_points = [(20, 400), (1080, 400)]

# Video writer
video_writer = cv2.VideoWriter("objects_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    count_reg_color=(175, 143, 111),
    count_bg_color=(164, 221, 237),
    reg_pts=line_points,
    line_thickness=2,
    track_thickness=1,
    names=model.names,
    draw_tracks=True,
    view_in_counts=True,
    view_out_counts=True,
    region_thickness=1,
    cls_txtdisplay_gap=1
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False)

    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

Line Counter Initiated.

0: 384x640 11 cars, 1 truck, 271.9ms
Speed: 0.0ms preprocess, 271.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 285.6ms
Speed: 1.5ms preprocess, 285.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 290.7ms
Speed: 0.0ms preprocess, 290.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 280.9ms
Speed: 0.0ms preprocess, 280.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 273.0ms
Speed: 0.0ms preprocess, 273.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 297.3ms
Speed: 0.0ms preprocess, 297.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 262.1ms
Speed: 0.0ms preprocess, 262.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 t

In [51]:
import cv2

from ultralytics import YOLO, solutions

model = YOLO("yolov8l.pt")

cap = cv2.VideoCapture("video.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

out = cv2.VideoWriter("areas_plot.avi", cv2.VideoWriter_fourcc(*"MJPG"), fps, (w, h))

analytics = solutions.Analytics(
    type="area",
    writer=out,
    im0_shape=(w, h),
    view_img=True,
    title='Frequency of Objects',
    y_label='Frequency',
    x_label='Frame',
    line_width=4,
    fontsize=20,
    
)

clswise_count = {}
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()
    if success:
        frame_count += 1
        results = model.track(frame, persist=True, verbose=True)

        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu()
            clss = results[0].boxes.cls.cpu().tolist()

            for box, cls in zip(boxes, clss):
                if model.names[int(cls)] in clswise_count:
                    clswise_count[model.names[int(cls)]] += 1
                else:
                    clswise_count[model.names[int(cls)]] = 1

        analytics.update_area(frame_count, clswise_count)
        clswise_count = {}
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 11 cars, 1 truck, 270.4ms
Speed: 0.0ms preprocess, 270.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 333.1ms
Speed: 2.0ms preprocess, 333.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 278.3ms
Speed: 0.0ms preprocess, 278.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 293.8ms
Speed: 0.0ms preprocess, 293.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 255.7ms
Speed: 0.0ms preprocess, 255.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 301.7ms
Speed: 0.0ms preprocess, 301.7ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 230.2ms
Speed: 1.0ms preprocess, 230.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 262.5ms
Speed: 0.

In [48]:
import cv2

from ultralytics import YOLO, solutions

model = YOLO("yolov8l.pt")
cap = cv2.VideoCapture("video.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("heatmap_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init heatmap
heatmap_obj = solutions.Heatmap(
    colormap=cv2.COLORMAP_PARULA,
    view_img=True,
    shape="circle",
    names=model.names,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False)

    im0 = heatmap_obj.generate_heatmap(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

None

0: 384x640 11 cars, 1 truck, 264.9ms
Speed: 0.0ms preprocess, 264.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 270.7ms
Speed: 2.0ms preprocess, 270.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 242.8ms
Speed: 2.0ms preprocess, 242.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 297.9ms
Speed: 0.0ms preprocess, 297.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 310.7ms
Speed: 1.1ms preprocess, 310.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 252.6ms
Speed: 10.2ms preprocess, 252.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 253.0ms
Speed: 1.1ms preprocess, 253.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 312.5ms
Spee